In [1]:
# Import the pandas library, which is the industry standard for structured data analysis.
# We assign it the alias 'pd' so we don't have to type 'pandas' every time.
import pandas as pd

In [2]:
file_path = 'luxury_cosmetics_fraud_analysis_2025.csv' #file path
df = pd.read_csv(file_path) #reads the file
print(f"Total Transactions: {len(df):,}") #tells us the number of transcations

Total Transactions: 2,133


In [3]:
# 1. Look at the first 5 rows to visually confirm the data looks right
print("\n--- First 5 Rows (df.head()) ---")
print(df.head())

# 2. Get a summary of the DataFrame: column types and null counts (the most important part of Day 1!)
print("\n--- Data Structure & Integrity (df.info()) ---")
df.info()


--- First 5 Rows (df.head()) ---
                         Transaction_ID                           Customer_ID  \
0  702bdd9b-9c93-41e3-9dbb-a849b2422080  119dca0b-8554-4b2d-9bec-e964eaf6af97   
1  2e64c346-36bc-4acf-bc2b-8b0fdf46abc5  299df086-26c4-4708-b6d7-fcaeceb14637   
2  29ad1278-70ce-421f-8d81-23816b39f4ac  dfa3d24d-b935-49a5-aa1d-7d57a44d8773   
3  07dc4894-e0eb-48f1-99a7-1942b1973d9b  7a67e184-9369-49ee-aeac-18f5b51b230f   
4  ae407054-5543-429c-918a-cdcc42ea9782  cf14730a-8f5a-453d-b527-39a278852b27   

  Transaction_Date Transaction_Time  Customer_Age Customer_Loyalty_Tier  \
0       2025-07-27         04:04:15          56.0                Silver   
1       2025-03-14         20:23:23          46.0              Platinum   
2       2025-02-20         12:36:02          32.0                Silver   
3       2025-04-25         19:09:43          60.0                Bronze   
4       2025-04-17         14:23:23           NaN              Platinum   

        Location           S

In [4]:
import re # import rename and regex expressions library

new_cols = df.columns.tolist() # for rename the time column using its index (3) because the time clumn isnt working by itself
new_cols[3] = 'Time_Raw'
df.columns = new_cols

df['Time_Clean'] = df['Time_Raw'].astype(str).str.replace(r'[^\d:]',' ', regex=True) # used to clean the string using rehex
# This replaces any character that is NOT a number (0-9) or a colon (:) with an empty string, 
# stripping out any stray letters, periods, or other garbage (like the problematic '.0' or extra digits).

df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date']) # convert transc date to datetime object
df['Transaction_Datetime'] = (df['Transaction_Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time_Clean']) #combine the date and the new cleaned time string

df['Transaction_Datetime'] = pd.to_datetime(df['Transaction_Datetime'], format='%Y-%m-%d %H:%M:%S',errors='coerce') # New safety feature: if conversion still fails, Pandas sets the value to NaT (Not a Time), instead of crashing.
df =df.drop(columns=['Time_Raw', 'Time_Clean'], errors='ignore')

print("Date/Time conversion successful.")
print(df[['Transaction_Date', 'Transaction_Datetime']].head())

Date/Time conversion successful.
  Transaction_Date Transaction_Datetime
0       2025-07-27  2025-07-27 04:04:15
1       2025-03-14  2025-03-14 20:23:23
2       2025-02-20  2025-02-20 12:36:02
3       2025-04-25  2025-04-25 19:09:43
4       2025-04-17  2025-04-17 14:23:23


In [5]:
df.columns = df.columns.str.strip() # takes out whitespace from every column name. to stop keyerrors
print("checking for column list spaces:", df.columns.tolist())

checking for column list spaces: ['Transaction_ID', 'Customer_ID', 'Transaction_Date', 'Customer_Age', 'Customer_Loyalty_Tier', 'Location', 'Store_ID', 'Product_SKU', 'Product_Category', 'Purchase_Amount', 'Payment_Method', 'Device_Type', 'IP_Address', 'Fraud_Flag', 'Footfall_Count', 'Transaction_Datetime']


In [6]:
median_age = df['Customer_Age'].median() #extracting the median from the age column
df['Customer_Age'] = df['Customer_Age'].fillna(median_age, inplace=True) #fills that column with the medians in places where "true" (there is a null or nan)

print(f"Cusomter_Age gaps filled with median age: {median_age:.1f}")

Cusomter_Age gaps filled with median age: 42.0


C:\Users\User\AppData\Local\Temp\ipykernel_9228\2473735203.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Customer_Age'] = df['Customer_Age'].fillna(median_age, inplace=True) #fills that column with the medians in places where "true" (there is a null or nan)


In [7]:
df['Customer_Loyalty_Tier'] = df['Customer_Loyalty_Tier'].fillna('None') # Fill the missing tiers with None to track them in a seperate risk category
print(df['Customer_Loyalty_Tier'].value_counts())

Customer_Loyalty_Tier
Bronze      808
Silver      506
Gold        423
Platinum    179
VIP         111
None        106
Name: count, dtype: int64


In [8]:
df['Transaction_Hour_of_Day'] = df['Transaction_Datetime'].dt.hour # extract the hour from datetime column -- 0=midnight 23= 11pm
print("\nTop 5 Transaction Hours:")
print(df['Transaction_Hour_of_Day'].value_counts().head()) #prints top 5 most common (head) hours to verify


Top 5 Transaction Hours:
Transaction_Hour_of_Day
22    106
12    104
15    104
2     102
17    102
Name: count, dtype: int64


In [9]:
df['Transaction_Day_of_Week'] = df['Transaction_Datetime'].dt.dayofweek # extract day of week -- monday = 0 sunday = 6
day_map = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
df['Transaction_Day_of_Week_Name'] = df['Transaction_Day_of_Week'].map(day_map) #mapping it to the name for better readability in reports
print("\nTransaction Volume by Day:")
print(df['Transaction_Day_of_Week_Name'].value_counts())


Transaction Volume by Day:
Transaction_Day_of_Week_Name
Tue    317
Sat    311
Thu    308
Wed    305
Mon    305
Fri    298
Sun    289
Name: count, dtype: int64


In [10]:
df.head()

,Transaction_ID,Customer_ID,Transaction_Date,Customer_Age,Customer_Loyalty_Tier,Location,Store_ID,Product_SKU,Product_Category,Purchase_Amount,Payment_Method,Device_Type,IP_Address,Fraud_Flag,Footfall_Count,Transaction_Datetime,Transaction_Hour_of_Day,Transaction_Day_of_Week,Transaction_Day_of_Week_Name
0,702bdd9b-9c93-41e3-9dbb-a849b2422080,119dca0b-8554-4b2d-9bec-e964eaf6af97,2025-07-27,None,Silver,San Francisco,FLAGSHIP-LA,NEBULA-SERUM-07,Concealer,158.24,Mobile Payment,Desktop,239.249.58.237,0,333,2025-07-27 04:04:15,4,6,Sun
1,2e64c346-36bc-4acf-bc2b-8b0fdf46abc5,299df086-26c4-4708-b6d7-fcaeceb14637,2025-03-14,None,Platinum,Zurich,BOUTIQUE-SHANGHAI,STELLAR-FOUND-03,Lipstick,86.03,Credit Card,Tablet,84.49.227.90,0,406,2025-03-14 20:23:23,20,4,Fri
2,29ad1278-70ce-421f-8d81-23816b39f4ac,dfa3d24d-b935-49a5-aa1d-7d57a44d8773,2025-02-20,None,Silver,Milan,POPUP-TOKYO,SOLAR-BLUSH-04,Mascara,255.69,Gift Card,Desktop,79.207.35.55,0,96,2025-02-20 12:36:02,12,3,Thu
3,07dc4894-e0eb-48f1-99a7-1942b1973d9b,7a67e184-9369-49ee-aeac-18f5b51b230f,2025-04-25,None,Bronze,London,BOUTIQUE-NYC,GALAXIA-SET-08,Serum,282.76,Gift Card,Mobile,176.194.167.253,0,186,2025-04-25 19:09:43,19,4,Fri
4,ae407054-5543-429c-918a-cdcc42ea9782,cf14730a-8f5a-453d-b527-39a278852b27,2025-04-17,None,Platinum,Miami,BOUTIQUE-NYC,LUNAR-MASC-02,Serum,205.86,Gift Card,Mobile,166.31.46.111,0,179,2025-04-17 14:23:23,14,3,Thu


In [11]:
# 1. Sort by Customer and Time (essential for the math to work)
df = df.sort_values(by=['Customer_ID', 'Transaction_Datetime'])

# 2. Calculate the time difference between consecutive transactions for each customer
df['Time_Since_Last_TXN'] = df.groupby('Customer_ID')['Transaction_Datetime'].diff()

# 3. Convert that time difference into total seconds (easier for Power BI to graph)
df['Time_Since_Last_TXN_Sec'] = df['Time_Since_Last_TXN'].dt.total_seconds()

# 4. Handle the "First Transaction" problem
# The very first time a customer shops, there is no "previous" transaction (NaN).
# We fill this with a large number (e.g., 30 days in seconds) to show they are "Cold."
df['Time_Since_Last_TXN_Sec'] = df['Time_Since_Last_TXN_Sec'].fillna(2592000) 

print("Velocity Features Created.")
print(df[['Customer_ID', 'Transaction_Datetime', 'Time_Since_Last_TXN_Sec']].head(10))

✅ Velocity Features Created.
                               Customer_ID Transaction_Datetime  \
210   001d3772-360f-455a-b16d-0faf018d9470  2025-05-27 11:44:15   
43    008348c6-138b-4231-8a28-8386f69c24e8  2025-03-10 01:53:07   
1399  00ac49cb-23ea-46d3-bd56-660d16ee9974  2025-08-09 00:15:02   
1479  00bdc439-e5dc-4ca3-a3da-b1eccbfecce4  2025-06-11 14:00:12   
317   00c1bab9-a602-452f-ba78-48e7f0eade27  2025-05-05 09:24:29   
878   00c5012b-02d6-4597-8c8b-914cb84181af  2025-07-01 15:12:06   
1065  00eafe7e-7764-4574-ae39-777710290a61  2025-04-19 19:03:38   
1637  0151b1aa-7f4c-4fb8-b363-322cc4a77a5b  2025-06-25 19:14:50   
895   01551dd3-0892-4dd7-8192-18f765c56e5e  2025-05-02 18:14:35   
1633  01741a26-634f-4183-a1bd-99ff5c69748d  2025-06-07 15:34:00   

      Time_Since_Last_TXN_Sec  
210                 2592000.0  
43                  2592000.0  
1399                2592000.0  
1479                2592000.0  
317                 2592000.0  
878                 2592000.0  
1065     

In [12]:
# 1. Customer-Level Deviation: Is this person spending way more than usual?
customer_avg = df.groupby('Customer_ID')['Purchase_Amount'].transform('mean')
df['Customer_Avg_Amount'] = customer_avg
df['Customer_Amount_Deviation'] = df['Purchase_Amount'] - df['Customer_Avg_Amount']

# 2. Category-Level Deviation: Is this item way more expensive than the category average?
category_avg = df.groupby('Product_Category')['Purchase_Amount'].transform('mean')
df['Category_Avg_Amount'] = category_avg
df['Category_Amount_Deviation'] = df['Purchase_Amount'] - df['Category_Avg_Amount']

# 3. Clean up the Datetime for SQL (SQL prefers strings if the library doesn't auto-convert)
# We make sure Transaction_Datetime is a clean string format
df['Transaction_Datetime_STR'] = df['Transaction_Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

print("Final Feature Engineering Complete.")
print(f"Total Columns now: {len(df.columns)}")

✅ Final Feature Engineering Complete.
Total Columns now: 26


In [13]:
import sqlite3

# 1. Connect to (or create) the database file
conn = sqlite3.connect('Luxury_Fraud_Data.db')

# 2. Write the DataFrame to a table named 'fraud_analysis'
# If the table already exists, we replace it ('replace')
df.to_sql('fraud_analysis', conn, if_exists='replace', index=False)

# 3. Verify it worked by running a quick SQL query
query = "SELECT COUNT(*) FROM fraud_analysis"
result = pd.read_sql_query(query, conn)

print(f" SUCCESS! The database is ready.")
print(f"Total rows loaded into SQL: {result.iloc[0,0]}")

# Close the connection
conn.close()

 SUCCESS! The database is ready.
Total rows loaded into SQL: 2133


C:\Users\User\AppData\Local\Temp\ipykernel_9228\1380797256.py:8: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  df.to_sql('fraud_analysis', conn, if_exists='replace', index=False)


In [14]:
# 1. Drop the column that SQLite doesn't like 
# We keep 'Time_Since_Last_TXN_Sec' because it's a standard number!
if 'Time_Since_Last_TXN' in df.columns:
    df = df.drop(columns=['Time_Since_Last_TXN'])

# 2. Re-connect and overwrite the table
conn = sqlite3.connect('Luxury_Fraud_Data.db')
df.to_sql('fraud_analysis', conn, if_exists='replace', index=False)
conn.close()

print("Database update, messy 'timedelta' column has been removed.")


Database update, messy 'timedelta' column has been removed.
